In [1]:
import dotenv
dotenv.load_dotenv(override=True)

import os
from typing import List
from agno.agent import Agent
from agno.tools.mcp import MCPTools
from agno.storage.json import JsonStorage
from agno.models.litellm import LiteLLM
from agno.models.openai.like import OpenAILike

import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Initialize LiteLLM/ OpenAI-like model

# model = LiteLLM(
#     id = "openai/gemma3:27b-it-qat", # provider need to be added before model name
#     api_base="https://ml.gss.com.tw/gemma3",
#     api_key="sk-test-key",
# )

model = OpenAILike(
    id = "openai/gemma3:27b-it-qat", # provider need to be added before model name
    base_url=os.getenv("AGENT_BASE_URL"),
    api_key=os.getenv("AGENT_API_KEY")
)

In [ ]:
# MCP server
rag_mcp_command = f"python3 /Agent_Framework_Demo/demo/1_rag_server.py stdio"
rag_mcp_server = MCPTools(
    command=rag_mcp_command,
    env={
        "OPENAI_BASE_URL": os.getenv("OPENAI_BASE_URL"),
        "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
    },
    timeout_seconds=60,
)

In [9]:
# structured output
from pydantic import BaseModel

class Reference(BaseModel):
    #relevant: bool
    score: float
    text: str

class RAGFormat(BaseModel):
    reference: List[Reference]
    final_answer: str

In [10]:
# multi-turn conversation
async def multi_turn_conversation(start_agent, user_id, session_id):
    while True:
        user_input = input("You: ")
        print(f"You: {user_input}")

        if user_input == "exit":
            print("Agent: Goodbye!")
            return result

        result = await start_agent.arun(
            message = user_input,
            user_id = user_id,
            session_id = session_id,
        )
        print(f"Agent: {result.content}")
    

In [11]:
# Agent
async def main(mcp_server, user_id, session_id):
    async with mcp_server as mcp_tools:
        seeker_agent = Agent(
            model = model,
            instructions = (
                "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
                "Behavior Rules: \n"
                "1. Direct Answering: If the question is clear and within your knowledge, answer directly.\n"
                "2. Retrieval-Based Answering: If the question requires specialized or external knowledge, retrieve relevant documents from the specific vector database. When retrieving from the database, the user's original intent should be preserved as much as possible, and the clarity of the question's meaning should be maintained.\n"
                "3. Clarification: If the question is vague or unclear, ask clarifying questions to understand the user’s intent before responding.\n"
            ),

            # tool-use settings
            tools = [mcp_tools],
            show_tool_calls=True,

            # history messages settings
            storage=JsonStorage(dir_path="agno_storage"),
            add_history_to_messages=True,
            num_history_runs=3,
            
            # structured output settings
            response_model=RAGFormat,
            parse_response=False, # if True, return Pydantic model
        )
        result = await multi_turn_conversation(seeker_agent, user_id, session_id)
        return result

In [12]:
result = asyncio.run(main(rag_mcp_server, user_id="user_1", session_id="session_1"))

You: 今天天氣真好～臺北現在33度
Agent: reference=[] final_answer='是啊，今天臺北天氣確實很好！33度是個熱情的好天氣呢。請問您有什麼需要我幫忙的嗎？'
You: 我想要請假出去玩，有沒有什麼限制？
Agent: reference=[Reference(score=0.7, text='此為依據勞動基準法所訂定的最基準工作時間。\n一、每週工作時數不得超過四十小時。\n二、每日工作時數為八小時。\n三、每日工作時間為\n08:30 開始上班\n12:00 午休及午餐時間\n13:00 下午工作時間開始\n15:00 下午休息時間\n15:15 繼續工作\n17:30 下班\n子女未滿一歲須員工親自哺乳者，除規定之休息時間外，本公司將每日另給哺乳時間二次，每次以三十分鐘為度，哺乳時間，視為工作時間。\n員工為撫育未滿三歲子女，得請求下列所定事項之一：\n1.\t每天減少工作時間一小時；減少之工作時間，不得請求報酬。\n2.\t調整工作時間。\n員工為前二項哺乳時間、減少或調整工時之請求時，本公司不得拒絕或視為缺勤而影響其全勤獎金、考績或為其他不利之處分。'), Reference(score=0.2645357, text='一、休假種類包含：特休假、生日假、普通傷病假、事假、家庭照顧假、婚假、喪假、產假、陪產假、產檢假、安胎假、公傷假、生理假、公假共十四種。\n二、請假方法：員工因故必須請假者，應事先填寫請假單或口頭敘明理由經核定後方可離開工作崗位或不出勤。\n \n請假天數，核准主管權限及事前天數辦法如下：\n請假天數\t事前天數\t核准主管權限\n0<天數≦5\t三天前\t部門主管\n5<天數≦10\t一週前\t處級主管\n天數>10\t兩週前\t業務/事業群總經理\n三、臨時狀況：同仁因突發事件或生病必須於當日請假者，得於當日10:00前以電話向主管報備，並於事後補辦請假手續。\n四、同仁未依規定辦理請假手續者，事後又不補辦請假手續並說明原因者，視同曠職。')] final_answer='您好，公司請假規定如下：\n\n*   **休假種類：** 公司有特休假、事假、婚假等等共十四種休假種類。\n*   **請假流程：** 需要事先填寫請假單或口頭敘明理由，經主管核定。\n*   **事前申請天數：**\n    *